In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain_teddynote import logging

logging.langsmith('RAG-Basic-PDF')

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG-Basic-PDF


# RAG 프로세스
### 사전단계
1. 문서 로드(Load)
2. 분할(Split)
3. 임베딩(Embedding)
4. 벡터DB(VectorStore)

### 수행단계
5. 검색(Retrieval)
6. 프롬프트(Prompt)
7. 모델(LLM)
8. 결과(Output)

In [ ]:
# !pip install langchain-community

  Using cached aiohttp-3.12.14-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.3-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp311-cp311-win_amd64.whl.metadata (76 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
   --


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# !pip install pymupdf

   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB 4.3 MB/s eta 0:00:05
   - -------------------------------------- 0.6/18.7 MB 5.8 MB/s eta 0:00:04
   -- ------------------------------------- 1.1/18.7 MB 6.9 MB/s eta 0:00:03
   --- ------------------------------------ 1.5/18.7 MB 7.4 MB/s eta 0:00:03
   ---- ----------------------------------- 2.0/18.7 MB 8.6 MB/s eta 0:00:02
   ----- ---------------------------------- 2.5/18.7 MB 8.7 MB/s eta 0:00:02
   ------ --------------------------------- 3.0/18.7 MB 9.1 MB/s eta 0:00:02
   ------- -------------------------------- 3.6/18.7 MB 9.5 MB/s eta 0:00:02
   -------- ------------------------------- 4.2/18.7 MB 9.9 MB/s eta 0:00:02
   ---------- ----------------------------- 4.8/18.7 MB 9.9 MB/s eta 0:00:02
   ----------- ---------------------------- 5.2/18.7 MB 10.1 MB/s eta 0:00:02
   ---------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# !pip install faiss-cpu

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.0 MB 2.8 MB/s eta 0:00:06
   - -------------------------------------- 0.5/15.0 MB 5.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.9/15.0 MB 5.7 MB/s eta 0:00:03
   --- ------------------------------------ 1.4/15.0 MB 6.8 MB/s eta 0:00:02
   ----- ---------------------------------- 2.0/15.0 MB 7.8 MB/s eta 0:00:02
   ------ --------------------------------- 2.5/15.0 MB 8.1 MB/s eta 0:00:02
   ------- -------------------------------- 3.0/15.0 MB 8.7 MB/s eta 0:00:02
   --------- ------------------------------ 3.5/15.0 MB 9.0 MB/s eta 0:00:02
   ----------- ---------------------------- 4.1/15.0 MB 9.4 MB/s eta 0:00:02
   ------------ --------------------------- 4.6/15.0 MB 9.4 MB/s eta 0:00:02
   ------------- -------------------------- 5.1/15.0 MB 9.9 MB/s eta 0:00:02
   ----------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [31]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data\SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(docs[1].page_content)
docs[1].metadata

2023년 12월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제 
   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1
   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2
   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3
   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4
   ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5
   ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6
 
 2. 기업/산업 
   ▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································ 7
   ▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11
   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': 'data\\SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data\\SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'modDate': "D:20231208132838+09'00'",
 'creationDate': "D:20231208132838+09'00'",
 'page': 1}

In [ ]:
# 단계 2: 텍스트 분할(Split Text)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents= text_splitter.split_documents(docs)

In [33]:
len(split_documents)

43

In [36]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [ ]:
# 단계 4: 벡터 스토어 생성(Create Vector Store)
# 메모리 공간에 존재
vector_store = FAISS.from_documents(split_documents, embeddings)

In [40]:
# 단계 5: 검색기(Retriever) 생성
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000294F8D87E90>, search_kwargs={})

In [54]:
# print(retriever.invoke("삼성전자가 개발한 생성AI는 이름은 무엇인가요?"))

In [55]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
You must include 'page' number in your answer.
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [56]:
# 단계 7: LLM 생성(Create LLM)
llm= ChatOpenAI(
    model="gpt-4o", temperature=0)

In [57]:
# 단계 8: 체인(chain) 생성
chain =(
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [58]:
from langchain_teddynote.messages import stream_response

In [59]:
question = "구글이 앤스로픽에 얼마를 투자했나요?"
response = chain.invoke(question)
stream_response(response)

구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자했습니다. (페이지 13)

제대로 된 답변이 출력되지 않는 경우 
1. LangSmith를 확인하여 검색기가 제대로 된 문서를 검색했는지 확인 (다른 Retriever를 사용해야할 수 있음)
2. 문서로드는 적절하게 발생하였는지 확인